# Analyses Of Vertices

In [1]:
import pickle
import sys
import numpy as np
sys.path.append('../')
import ScriptExtract
with open('simple_auto_table.pickle', 'rb') as f:
    table = pickle.load(f)
    f.close()

In [2]:
table['1.txt'][0]

{'Sentence': 'Проблемы передвижения только пешком или на общественном транспорте возникли после окончания университета.',
 'Actions': [<ScriptExtract.Preprocessing.action.action at 0x7f0a28f2fe10>,
 'TagUd': [],
 'SecondLevel': 0,
 'FirstLevel': 1,
 'Relations': [{'tp': 'медиатив',
   'parent': {'start': 67, 'end': 75},
   'child': {'start': 56, 'end': 66}},
  {'tp': 'предикат',
   'parent': {'start': 67, 'end': 75},
   'child': {'start': 9, 'end': 21}}]}

In [3]:
from ScriptExtract.GraphScript import graph_construction
import imp
imp.reload(graph_construction)
(E, V), (start, end), (full_list_actions, verb_dict, feature_dict) = graph_construction.construct_graph(table, key_word = 'sem_rel')

SyntaxError: invalid syntax (graph_construction.py, line 121)

In [4]:
graph_construction.graph_inform(V,E)

The edges number 74
The out-edges number 74
The vertices number 23
The number of document 3


In [5]:
def BFS(E = E, start = start, end = end):
    visited = {v:False for v in E}
    queue = [([],start)]
    while(len(queue)):
        path, v = queue.pop(-1)
        if end in E[v]:
            return path +[v, end]
        next_vertices = [(path + [v], i) for i in E[v] if not visited[i]]
        queue.extend(next_vertices)
        for v in next_vertices:
            visited[v[1]]= True
    return None
BFS()

In [6]:
docs = np.unique([i[0] for i in V])
d = np.argmax(np.array([len([i for i in V if i[0]==j]) for j in docs if j>=0]))
v_new = [i for i in V if i[0]==d]

In [7]:
for v in v_new:
    act = full_list_actions[v[0]][v[1]]
    print(' '.join(act.sentence))
    print()
    print('VERB: ' + act.inform['VERB'][0].lemma)
    print()
    for j in feature_dict:
        if v in feature_dict[j]:
            print(feature_dict[j][v] + ": " + j)
    print()
    print(act.inform['SEM_REL'])
    for j in act.inform['SEM_REL']:
        print(j['tp'])
        print(' '.join(act.sentence)[j['parent']['start']:j['parent']['end']])
    print("="*100 + '\n')

Я посчитал программу интересной .

VERB: посчитать

авторизатор: я
объект: программа

[{'tp': 'авторизатор', 'parent': {'start': 4, 'end': 12}, 'child': {'start': 2, 'end': 3}}, {'tp': 'объект', 'parent': {'start': 4, 'end': 12}, 'child': {'start': 13, 'end': 22}}]
авторизатор
считал п
объект
считал п

Государственная программа « первый автомобиль » предоставляет возможность покупки автомобиля с 10 % скидкой при условии стоимости до миллиона рублей и сборке в РФ .

VERB: предоставлять

субъект: программа

[{'tp': 'объект', 'parent': {'start': 48, 'end': 61}, 'child': {'start': 36, 'end': 46}}, {'tp': 'субъект', 'parent': {'start': 48, 'end': 61}, 'child': {'start': 18, 'end': 27}}]
объект
предоставляет
субъект
предоставляет

Я рассматривал следующие параметры : мощность двигателя , объем и удобство салона , наличие полезных функций , внешний вид авто .

VERB: рассматривать

объект: параметр

[{'tp': 'авторизатор', 'parent': {'start': 4, 'end': 16}, 'child': {'start': 38, 'end': 46}}, {

In [271]:
import copy
def equal_word(word, word1):
    return word.begin == word1['start'] and word.end == word1['end']
class script:
    def __init__(self, V, E, full_list_actions, start = (-1,-1), end = (-2,-2)):
        self.V, self.E = V, E
        self.start = start
        self.end = end
        self.V_inform, self.E_inform = self._reconstruct(V, E, full_list_actions)
        self.V_descr = {}
        self.bad_BFS()

    def _reconstruct(self, V, E, full_list_actions):
        E = E
        V = {v: self._get_inform(v, full_list_actions) for v in V}
        return V, E
    
    def bad_BFS(self):
        start, end = self.start, self.end
        self.V.sort(key = lambda x: x[0])
        self.V = [i for i in self.V if i!= start and i!=end]
        self.V = [start] + self.V + [end]
        self.V_descr[start] = {}
        previous = start
        for v in self.V[1:]:
            if v in self.E and len(self.E[v]) >0 or v == end:
                self.V_descr[v] = copy.deepcopy(self.V_descr[previous])
                self.update(v)
                #print(v)
                previous = v
        return None
    
    def update(self, v):
        inform = self.V_inform[v]
        print(inform.keys())
        if 'Sentence' in inform:
            sentence = inform['Sentence']
            self.V_descr[v]['Sentence'] = sentence
        else:
            self.V_descr[v]['Sentence'] = ''
        if 'локатив' in inform:
            self.V_descr[v]['локатив'] = {'локатив':inform['локатив'],
                                          'Sentence':sentence}
        if 'объект' in inform:
            obj = {'объект':inform['объект'],
                    'verb':inform['VERB'],
                  'Sentence':sentence}
            if 'объект' in self.V_descr[v]:
                self.V_descr[v]['объект'].append(obj)
            else:
                self.V_descr[v]['объект'] = [obj]
        if 'субъект' in inform:
            subj = {'субъект':inform['субъект'],
                    'verb':inform['VERB'],
                   'Sentence':sentence}
            if 'субъект' in self.V_descr[v]:
                self.V_descr[v]['субъект'].append(subj)
            else:
                self.V_descr[v]['субъект'] = [subj]
        if 'темпоратив' in inform:
            self.V_descr[v]['темпоратив'] = {'темпоратив':inform['темпоратив'],
                                          'Sentence':sentence}
        
    def _get_inform(self, v, full_list_actions):
        if v == self.start or v == self.end:
            return {}
        act = full_list_actions[v[0]][v[1]]
        inform = {}
        inform['Sentence'] = act.sentence
        inform['VERB'] = act.inform['VERB']
        verb = act.inform['VERB'][0].lemma
        for j in act.inform['SEM_REL']:
            type_rel = j['tp']
            parent = j['parent']
            child = j['child']
            if equal_word(act.inform['VERB'][0], parent):
                for key in act.inform:
                    if not key in ['VERB', 'SEM_REL']:
                        for word, list_depend, _ in act.inform[key]:
                            if equal_word(word, child):
                                if type_rel in inform:
                                    inform[type_rel].append((word, list_depend))
                                else:
                                    inform[type_rel] = [(word, list_depend)]
        return inform
    
    def GetNext(self, v):
        return self.E[v]

In [272]:
script = script(V,E, full_list_actions)

dict_keys(['Sentence', 'VERB', 'авторизатор', 'объект'])
dict_keys(['Sentence', 'VERB', 'субъект'])
dict_keys(['Sentence', 'VERB', 'объект'])
dict_keys(['Sentence', 'VERB', 'транзитив'])
dict_keys(['Sentence', 'VERB', 'делибератив', 'субъект'])
dict_keys(['Sentence', 'VERB', 'объект', 'субъект'])
dict_keys(['Sentence', 'VERB', 'локатив', 'объект', 'субъект'])
dict_keys(['Sentence', 'VERB', 'субъект'])
dict_keys(['Sentence', 'VERB', 'делибератив', 'субъект'])
dict_keys(['Sentence', 'VERB', 'авторизатор', 'объект', 'эстиматив'])
dict_keys(['Sentence', 'VERB', 'локатив', 'объект', 'субъект'])
dict_keys(['Sentence', 'VERB', 'локатив'])
dict_keys(['Sentence', 'VERB', 'директив', 'субъект'])
dict_keys(['Sentence', 'VERB', 'локатив', 'объект', 'субъект'])
dict_keys(['Sentence', 'VERB', 'локатив', 'субъект'])
dict_keys(['Sentence', 'VERB', 'директив', 'субъект'])
dict_keys(['Sentence', 'VERB', 'интерпрет_понятия', 'субъект'])
dict_keys(['Sentence', 'VERB', 'объект', 'потенсив', 'субъект'])
dic

In [273]:
V_inform = script.V_inform

In [274]:
def union(sentence, list_):
    q = list(np.array(sentence)[list_])
    return ' '.join(q)
for v in V_inform:
    if v[0]>=0:
        v = V_inform[v]
        sentence = v['Sentence']
        print(' '.join(sentence))
        print()
        print('VERB:', v['VERB'][0].lemma)
        for i in v:
            if not i in ['Sentence', 'VERB']:
                for j in v[i]:
                    print('%s: %s (%s)'%(i, j[0].lemma, union(sentence, j[1])))
        print('\n' + '='*100)

Я посчитал программу интересной .

VERB: посчитать
авторизатор: я (Я)
объект: программа (программу интересной)

Государственная программа « первый автомобиль » предоставляет возможность покупки автомобиля с 10 % скидкой при условии стоимости до миллиона рублей и сборке в РФ .

VERB: предоставлять
субъект: программа (Государственная программа первый автомобиль)

Я рассматривал следующие параметры : мощность двигателя , объем и удобство салона , наличие полезных функций , внешний вид авто .

VERB: рассматривать
объект: параметр (следующие параметры мощность двигателя объем и удобство салона наличие полезных функций внешний вид авто)

Я проверил рынок автомобилей , подходящих по этому параметру .

VERB: подходить
транзитив: параметр (по этому параметру)

После подсчета имеющихся средств , я рассмотрел варианты его покупки по госпрограмме и в кредит .

VERB: рассматривать
делибератив: вариант (варианты его покупки по госпрограмме и в кредит)
субъект: я (я)

Я разобрал варианты покупки как 

In [275]:
V_descr = script.V_descr

In [287]:
def print_dict(v_desr):
    if 'Sentence' in v_desr:
        print(' '.join(v_desr['Sentence']))
    if 'локатив' in v_desr:
        print('\nлокатив:'.upper())
        print("%s (%s)"%(v_desr['локатив']['локатив'][0][0].lemma,
                         union(v_desr['локатив']['Sentence'], v_desr['локатив']['локатив'][0][1])))
    if 'темпоратив' in v_desr:
        print('\nтемпоратив:'.upper())
        print("%s (%s)"%(v_desr['темпоратив'][0].lemma, union(v_desr['Sentence'], v_desr['темпоратив'][1])))
    if 'объект' in v_desr:
        print('\nобъект'.upper())
        obj = v_desr['объект']
        for i in obj:
            for j in i['объект']:
                print("%s (%s) - %s (%s)"%(j[0].lemma, union(i['Sentence'], j[1]),
                                           i['verb'][0].lemma, union(i['Sentence'], i['verb'][1])))
    if 'субъект' in v_desr:
        print('\nсубъект'.upper())
        obj = v_desr['субъект']
        for i in obj:
            for j in i['субъект']:
                print("%s (%s) - %s (%s)"%(j[0].lemma, union(i['Sentence'], j[1]),
                                           i['verb'][0].lemma, union(i['Sentence'], i['verb'][1])))
for v in script.V:
    print('Num Vert:', v)
    print_dict(V_descr[v])
    print('\n' + '='*100)

Num Vert: (-1, -1)

Num Vert: (0, 17)
Я посчитал программу интересной .

ОБЪЕКТ
программа (программу интересной) - посчитать (посчитал)

Num Vert: (0, 15)
Государственная программа « первый автомобиль » предоставляет возможность покупки автомобиля с 10 % скидкой при условии стоимости до миллиона рублей и сборке в РФ .

ОБЪЕКТ
программа (программу интересной) - посчитать (посчитал)

СУБЪЕКТ
программа (Государственная программа первый автомобиль) - предоставлять (предоставляет)

Num Vert: (0, 33)
Я рассматривал следующие параметры : мощность двигателя , объем и удобство салона , наличие полезных функций , внешний вид авто .

ОБЪЕКТ
программа (программу интересной) - посчитать (посчитал)
параметр (следующие параметры мощность двигателя объем и удобство салона наличие полезных функций внешний вид авто) - рассматривать (рассматривал)

СУБЪЕКТ
программа (Государственная программа первый автомобиль) - предоставлять (предоставляет)

Num Vert: (0, 20)
Я проверил рынок автомобилей , подходящих п

In [1]:
# The Script